### Env Config

In [22]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

In [23]:
# Aproximação da quantidade de linhas esperadas no arquivo final 
TOTAL_ROW_COUNT = 500000
# Randomizar amostras coletadas de cada arquivo
DO_RANDOMIZE_SAMPLE_ROWS = True

# Exportar os arquivos tratados ou não
DO_EXPORT_FILES = True

# Rodar ou não a visualização dos recursos (Demora muito mais tempo)
RUN_RESOURCE_VISUALIZATION = False

In [24]:
def export_to_file(df: DataFrame, path: str):
	if DO_EXPORT_FILES:
		if os.path.exists(f'{path}/df.csv'):
			os.remove(f'{path}/df.csv')
		if os.path.exists(f'{path}/df.parquet'):
			try:
				os.remove(f'{path}/df.parquet')
			except:
				os.system(f'rm -rf {path}/df.parquet')
		df.to_csv(f'{path}/df.csv', index=False)
		df.to_parquet(f'{path}/df.parquet', index=False)

### Criando dataframe

In [25]:
files = os.listdir('_data/raw/operacoes-credito')
files.sort()

sample_size = TOTAL_ROW_COUNT // len(files)

arrow_tables = []
for file in files:
	print(f'Processando o arquivo {file}')

	temp_df = pd.read_parquet(f'_data/raw/operacoes-credito/{file}', engine='pyarrow').sample(n=sample_size, random_state=random.randint(1, 1000))
	table = pa.Table.from_pandas(temp_df)
	arrow_tables.append(table)

combined_table = pa.concat_tables(arrow_tables)

df = combined_table.to_pandas()

Processando o arquivo 201206.parquet
Processando o arquivo 201207.parquet
Processando o arquivo 201208.parquet
Processando o arquivo 201209.parquet
Processando o arquivo 201210.parquet
Processando o arquivo 201211.parquet
Processando o arquivo 201212.parquet
Processando o arquivo 201301.parquet
Processando o arquivo 201302.parquet
Processando o arquivo 201303.parquet
Processando o arquivo 201304.parquet
Processando o arquivo 201305.parquet
Processando o arquivo 201306.parquet
Processando o arquivo 201307.parquet
Processando o arquivo 201308.parquet
Processando o arquivo 201309.parquet
Processando o arquivo 201310.parquet
Processando o arquivo 201311.parquet
Processando o arquivo 201312.parquet
Processando o arquivo 201401.parquet
Processando o arquivo 201402.parquet
Processando o arquivo 201403.parquet
Processando o arquivo 201404.parquet
Processando o arquivo 201405.parquet
Processando o arquivo 201406.parquet
Processando o arquivo 201407.parquet
Processando o arquivo 201408.parquet
P

### Ajuste de tipagem dos dados

In [26]:
df.dtypes

data_base                         object
uf                                object
cliente                           object
ocupacao                          object
cnae_secao                        object
porte                             object
modalidade                        object
numero_de_operacoes               object
carteira_ativa                    object
carteira_inadimplida_arrastada    object
ativo_problematico                object
indexador_modalidade              object
dtype: object

In [27]:
df['cliente'].value_counts()

cliente
PJ    430096
PF     69808
Name: count, dtype: int64

In [28]:
df = df.rename(columns={
	'cliente': 'classificacao',
	'cnae_secao': 'cnae',
})

In [29]:
if RUN_RESOURCE_VISUALIZATION:
	print(df['porte'].value_counts())
	print(df['modalidade'].value_counts())
	print(df['ocupacao'].value_counts())
	print(df['cnae'].value_counts())

In [30]:
df['porte'] = df['porte'].str.slice(5).str.strip()
df['modalidade'] = df['modalidade'].str.slice(5).str.strip()
df['ocupacao'] = df['ocupacao'].str.slice(5).str.strip()
df['cnae'] = df['cnae'].str.slice(5).str.strip()

In [31]:
MONETARY_COLS = ['carteira_ativa', 'carteira_inadimplida_arrastada', 'ativo_problematico']

CATEGORY_COLS = ['uf', 'ocupacao', 'cnae', 'porte', 'modalidade', 'indexador_modalidade', 'classificacao']

QUANTITY_COLS = ['numero_de_operacoes']

DATE_COLS = ['data_base']

for column in df.columns:
	if column in MONETARY_COLS:
		df[column] = df[column].str.replace(',', '.').astype(float)
		df = df.rename(columns={column: f'vl_{column}'})
	elif column in CATEGORY_COLS:
		df[column] = df[column].astype('category')
		df = df.rename(columns={column: f'ct_{column}'})
	elif column in QUANTITY_COLS:
		df[column] = df[column].str.replace('<= ', '').astype(int)
		df = df.rename(columns={column: f'qt_{column}'})
	elif column in DATE_COLS:
		df[column] = pd.to_datetime(df[column], format='%Y-%m-%d')
		df = df.rename(columns={column: f'dt_{column}'})

In [32]:
df.dtypes

dt_data_base                         datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
qt_numero_de_operacoes                        int64
vl_carteira_ativa                           float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
ct_indexador_modalidade                    category
dtype: object

### Exportando para zona Trusted

In [33]:
export_to_file(df, '_data/trusted/operacoes-credito')

In [34]:
df['vl_carteira_problematica'] = df['vl_ativo_problematico'] - df['vl_carteira_inadimplida_arrastada']

In [35]:
df['vl_carteira_problematica'].describe()

count       499904.00
mean        218862.15
std        6658520.63
min        -403751.54
25%              0.00
50%              0.00
75%           5104.27
max     2692394570.78
Name: vl_carteira_problematica, dtype: float64

#### Valores menores do que 0?

In [36]:
df[df['vl_carteira_problematica'] < 0]['vl_carteira_problematica']

91600      -2182.56
88508      -3548.83
434408      -459.61
88392       -332.26
398591      -459.34
91273     -30577.07
45580     -16030.40
45636      -4847.74
92409     -26231.26
161217   -308150.76
463577    -74283.36
337411    -16437.49
275588    -15286.59
360161    -13201.76
473264   -387782.90
292173    -36360.66
340283   -403751.54
223394    -21143.71
363300     -7138.51
162139   -130997.84
668193     -9742.33
161312      -247.17
584916     -5187.60
17275      -4212.64
17611      -3078.49
525649      -274.01
185243     -3101.81
784519    -10887.65
768103   -130140.91
588134     -4349.92
572112    -42516.54
232637     -6617.23
917764    -35583.68
571521    -11353.75
Name: vl_carteira_problematica, dtype: float64

#### 0.05% da amostra está danificada, vamos capar em 0

In [37]:
df['vl_carteira_problematica'] = df['vl_carteira_problematica'].clip(lower=0)

In [38]:
df['vl_carteira_problematica'].describe()

count       499904.00
mean        218865.68
std        6658520.44
min              0.00
25%              0.00
50%              0.00
75%           5104.27
max     2692394570.78
Name: vl_carteira_problematica, dtype: float64

In [39]:
df['vl_carteira_ativa'] = df['vl_carteira_ativa'] - df['vl_carteira_problematica']
df.rename(columns={ 'vl_carteira_ativa': 'vl_carteira_saudavel' }, inplace=True)

In [40]:
df['vl_carteira_saudavel'].describe()

count        499904.00
mean        5275615.50
std        97636953.39
min               0.00
25%           16263.89
50%           95371.20
75%          520099.16
max     22774644112.85
Name: vl_carteira_saudavel, dtype: float64

In [41]:
df.dtypes

dt_data_base                         datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
qt_numero_de_operacoes                        int64
vl_carteira_saudavel                        float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
ct_indexador_modalidade                    category
vl_carteira_problematica                    float64
dtype: object

### Exportando para zona Refined

In [42]:
export_to_file(df, '_data/refined/operacoes-credito')